In [ ]:
from importlib import reload
import platform, os, sys, datetime, re, itertools, warnings, pickle, bz2
from os.path import join
from glob import glob
import cv2
import numpy as np
from scipy.signal import find_peaks
import pandas as pd
import matplotlib.pyplot as plt
import tracker.utils as utils
from collections import defaultdict

tank_diameter_vs_age = { 7:9.6, 14:10.4, 21:12.8, 28:17.7, 42:33.8,
                         56:33.8, 70:33.8, 84:3.8 }
tank_radius_vs_age = { k:v/2 for k,v in tank_diameter_vs_age.items() }
# plt.rcParams['figure.dpi'] = 150
plt.rcParams['figure.figsize'] = 9,6

analysis_dir = './analysis'

# Prepare trial data.

Prepare a dictionary of trials to analyze with basic info for each (path to trial file, population, age, number of individuals, etc).

In [ ]:
# Define how to load a trial's tracking output.
def load_trial(trial_file,load_data=True):
    trial_dir  = os.path.dirname(trial_file)
    trial_name = os.path.basename(trial_dir)
    pop,day,age,group,n_ind = trial_name.split('_')[:5]
    pop        = {'sf':'SF', 'pa':'Pa', 'rc':'RC'}[pop.lower()]
    age        = int(age[:-3])
    age = 42 if age==43 else (70 if age==71 else age)
    n_ind      = int(re.findall('\d+',n_ind)[0])
    trial      = { k:v for k,v in locals().items() if k in ['trial_dir', 
                   'trial_name', 'pop', 'age', 'group', 'n_ind'] }
    trial['R_cm'] = tank_radius_vs_age.get(age,None)
    if load_data:
        with open(trial_file,'rb') as f:
            trial.update(pickle.load(f))
        ellipse = cv2.fitEllipse(trial['tank']['contour'])
        trial['center'] = np.array(ellipse[0])
        trial['R_px']   = np.mean(ellipse[1])/2
    return trial

# Select a set of trials to analyze.
trial_files = sorted(glob('../tracking/full_21-01-22/*/trial.pik'))

# Count trials of each type.
trials = [load_trial(f,load_data=False) for f in trial_files]
trials = pd.DataFrame(trials,index=trial_files)
grouped_trials = trials.groupby(['pop','age','n_ind'])
count  = pd.DataFrame(grouped_trials['trial_dir'].count().rename('count'))
count = count.unstack(1)
count.columns = count.columns.droplevel()
count[pd.isna(count)] = 0
count = count.astype(int)
print('Number of trials of each type:')
count

# Define analysis functions.

In [ ]:
# def dist2ellipse(semi_major, semi_minor, xy):
#     px,py = np.absolute(xy)
#     tx,ty = 0.707,0.707
#     a = semi_major
#     b = semi_minor
#     for x in range(0, 3):
#         x   = a * tx
#         y   = b * ty
#         ex  = (a*a - b*b) * tx**3 / a
#         ey  = (b*b - a*a) * ty**3 / b
#         rx  = x - ex
#         ry  = y - ey
#         qx  = px - ex
#         qy  = py - ey
#         r   = np.hypot(ry, rx)
#         q   = np.hypot(qy, qx)
#         tx  = min(1, max(0, (qx * r / q + ex) / a))
#         ty  = min(1, max(0, (qy * r / q + ey) / b))
#         t   = np.hypot(ty, tx)
#         tx /= t 
#         ty /= t 
#     return (np.copysign(a * tx, xy[0]), np.copysign(b * ty, xy[1]))    


def compute_kinematics(trial,wall_distance=False):
    center      = trial['center']
    px2cm       = trial['R_cm']/trial['R_px']
    n           = trial['n_ind']
    pos         = trial['data'][:,:n,:3].copy() # discard extra objects
    pos[:,:,:2] = (pos[:,:,:2]-center[None,None,:])*px2cm # convert to centimeters
    pos[:,:,1]  = -pos[:,:,1] # flip y axis
    for j in range(n): # unwrap orientations
        I          = ~np.isnan(pos[:,j,2])
        pos[I,j,2] = np.unwrap(pos[I,j,2])
    time        = trial['frame_list']/trial['fps']
    vel         = np.gradient(pos,time,axis=0)
    acc         = np.gradient(vel,time,axis=0)
    v           = np.hypot(vel[:,:,0],vel[:,:,1])
    
    d_wall      = trial['R_cm'] - np.hypot(pos[:,:,0],pos[:,:,1])
# #     dist    = lambda xy: dist2ellipse(*ellipse[1],xy)
#     dist    = lambda xy: cv2.pointPolygonTest(trial['tank']['contour'],tuple(xy),True)
#     d_wall  = px2cm * np.apply_along_axis(dist,2,trial['data'][:,:,:2])
    
    trial.update({ k:v for k,v in locals().items() if k in 
                   ['time', 'pos', 'vel', 'acc', 'd_wall', 'v'] })
    return trial


def compute_cuts(trial,ranges):
    # Distances in ut ranges
    globals().update(trial)
    
    # valid array: axis 0 = time, axis 1 = [nan_xy,nan_any,d_wall,v,v_ang,final]
    valid  = np.full(pos.shape[:2]+(7,),np.True_,dtype=np.bool_)
    valid[:,:,0] = np.logical_not(np.any(np.isnan(pos),axis=2))
    valid[:,:,1] = np.logical_not(np.any(np.isnan(vel),axis=2))
    valid[:,:,2] = np.logical_not(np.any(np.isnan(acc),axis=2))
    valid[:,:,3] = np.logical_and(d_wall>=ranges['d_wall'][0],d_wall<=ranges['d_wall'][1])
    valid[:,:,4] = np.logical_and(v>=ranges['v'][0],v<=ranges['v'][1])
    valid[:,:,5] = np.logical_and(vel[:,:,2]>=ranges['v_ang'][0],vel[:,:,2]<=ranges['v_ang'][1])
    valid[:,:,6] = np.all(valid[:,:,:5],axis=2)
    
    n_total = valid.shape[0]*valid.shape[1]
    n_valid = np.count_nonzero(valid,axis=(0,1))
    valid_fraction = { 'nan_pos' : n_valid[0]/n_total, 
                       'nan_vel' : n_valid[1]/n_total, 
                       'nan_acc' : n_valid[2]/n_total, 
                       'd_wall'  : n_valid[3]/n_valid[0], 
                       'v'       : n_valid[4]/n_valid[1], 
                       'v_ang'   : n_valid[5]/n_valid[1], 
                       'final'   : n_valid[6]/n_total     }
    
    trial.update({'valid':valid, 'valid_fraction':valid_fraction, 'cut_label':cut_label})
    return trial


# t0 = datetime.datetime.now()
# trial = load_trial(trial_files[0], load_data=True)
# trial = compute_kinematics(trial)
# trial = compute_cuts(trial)
# print(datetime.datetime.now()-t0)

### [in progress] Detect discrete turns.

In [ ]:
# for i,trial_file in enumerate(trials.index):
#     print('\r'+' '*200+'\r'+f'{i}/{len(trial_files)}',end='')
#     trial = load_trial(trial_file, load_data=True)
#     trial = compute_kinematics(trial)
#     locals().update(trial)
#     # Plot angle.
#     a = pos[:,0,2]
#     plt.plot(time,a,label='actual angle')
#     # Extract discrete turns.
#     w = int(0.15*fps)
#     step = np.concatenate([np.ones(w)/w,np.zeros(1),-np.ones(w)/w])
#     conv = np.convolve(a,step,mode='same')
#     P = find_peaks(np.absolute(conv),distance=w,height=np.pi/20)
#     # Reconstruct angle vs time using only the discrete turns.
#     da = np.zeros_like(a)
#     da[P[0]] = conv[P[0]]
#     a2 = a[0]+np.cumsum(da)
#     plt.plot(time,a2,label='reconstructed angle')
    
#     plt.plot(time,a-a2,label='actual angle - reconstructed angle')
    
# #     plt.xlim(0,1000)
# #     plt.ylim(2,15)
#     plt.legend()
    
#     break

In [ ]:
# fig,ax = plt.subplots(1,2,figsize=(12,4))
# I = P[0]
# dt = time[I[1:]]-time[I[:-1]]
# ax[0].hist(dt,bins=50)
# ax[0].set_xlabel('time between two discrete turns')
# ax[0].set_ylabel('frequency')
# ax[0].set_yscale('log')
# ax[1].hist(np.absolute(conv[I]),bins=50)
# ax[1].set_xlabel('angle turned')
# ax[1].set_yscale('log')
# plt.suptitle('Discrete turns')
# plt.show()


# fig,ax = plt.subplots(1,2,figsize=(12,4))
# da = np.diff(a-a2)/np.sqrt(time[1:]-time[:-1])
# ax[0].hist(da,bins=100)
# ax[0].set_xlabel('da/sqrt(dt)')
# ax[0].set_ylabel('frequency')
# # dt = time[I[1:]]-time[I[:-1]]
# # ax[0].hist(dt,bins=20)
# # ax[1].hist(np.absolute(conv[I]),bins=50)
# # plt.xlim(-5,5)
# ax[0].set_yscale('log')
# ax[1].set_visible(False)
# plt.suptitle('Continuous turns')
# plt.show()

# Compute or load analyzed data.

The first subsection iterates over the list of trials, loads the full tracking output, compute kinematic quantities, performs cuts, computes statistical properties (distribution of speed, angular speed, pairwise distance-angle, etc), and saves them.

The second subsection loads precomputed statistical properties made with the first cell.

### Define cuts.

In [ ]:
# Speed cut: 0.2*30 = travel 0.2 tank radius between 2 frames at 30fps.
# Angular speed cut: pi/2*30 = quarter turn between 2 frames at 30 fps.
cut_ranges    = { 'd_wall':[-np.inf,np.inf], 'v':[0,0.2*30], 
                  'v_ang':[-np.pi/2*30,np.pi/2*30] }
# This is used to name the output directory and make figure titles.
# The "=" should be "<=" however ntfs doesn't allow "<" in filenames.
cut_label     = ', '.join([ f'{v[0]:g}={k}={v[1]:g}' for k,v in cut_ranges.items() ])

cut_dir = os.path.join(analysis_dir,cut_label)
if not os.path.exists(cut_dir):
    os.mkdir(cut_dir)

### Analyze trials.

In [ ]:
bins_area     = np.linspace(0,600,100)
bins_aspect   = np.linspace(1,15,100)
prebins_dWall = np.linspace(-0.1,1.1,100) # Multiply by R_cm before using.
prebins_v     = np.linspace(*cut_ranges['v'],100) # Multiply by R_cm before using.
bins_vAng     = np.linspace(*cut_ranges['v_ang'],100)
prebins_pairDist = np.linspace(0,2,60) # Multiply by R_cm before using.
bins_pairAng  = np.linspace(0,np.pi,30)

trial_data    = {}

for i,trial_file in enumerate(trials.index):
    print('\r'+' '*200+'\r'+f'{i+1}/{len(trials)}',end='')
    
    trial  = load_trial(trial_file, load_data=True)
    ranges = { 'd_wall': [ x*trial['R_cm'] for x in cut_ranges['d_wall'] ], 
               'v':      [ x*trial['R_cm'] for x in cut_ranges['v'] ], 
               'v_ang':  cut_ranges['v_ang'] }
    trial  = compute_kinematics(trial)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        trial = compute_cuts(trial, ranges)
    globals().update(trial)
    
    # Fish area and Fish aspect ratio histograms.
    # Useful to tune the tracker's contour filters.
    hist_area   = np.histogram(data[:,:n_ind,3], bins=bins_area )
    hist_aspect = np.histogram(data[:,:n_ind,4], bins=bins_aspect )
    
    # Distribution of distance to the wall.
    bins = prebins_dWall * R_cm
    vals = d_wall.flatten()
    hist_dWall = np.histogram(vals[~np.isnan(vals)],bins=bins)
    
    # Speed distribution.
    bins = prebins_v * R_cm
    vals = v.flatten()
    hist_v = np.histogram(vals[~np.isnan(vals)],bins=bins)
    
    # Angular speed distribution.
    bins = bins_vAng
    vals = vel[:,:,2].flatten()
    hist_vAng = np.histogram(vals[~np.isnan(vals)],bins=bins)
    
    # Joint distribution of pair distance and pair angle,
    if n_ind>1:
        bins_d  = prebins_pairDist * R_cm
        bins_a  = bins_pairAng
        J1,J2 = np.triu_indices(n_ind,1)
        d     = np.hypot(pos[:,J1,0]-pos[:,J2,0],pos[:,J1,1]-pos[:,J2,1]).flatten()
        a     = (pos[:,J1,2]-pos[:,J2,2]).flatten()
        a     = a - 2*np.pi*np.rint(a/(2*np.pi))
        I     = np.logical_not(np.logical_or(np.isnan(d),np.isnan(a)))
        d     = d[I]
        a     = np.absolute(a[I])
        # 2D histogram of pairwise distance and angle.
        hist_distAng = np.histogram2d(d, a, bins=(bins_d,bins_a), density=True)
        # Pairwise polar alignment parameter vs pair distance.
        K = np.digitize(d,bins_d)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            p = np.array([ np.nanmean(np.cos(a[K==i])) for i in range(len(bins_d)+1) ])
        polar = p[1:-1],bins_d
    else:
        hist_distAng,polar = None,None

    # Save output.
    trial_data[trial_file] = { k:v for k,v in locals().items() if k in 
                               [ 'valid_fraction', 'hist_area', 'hist_aspect', 'hist_dWall', 
                                 'hist_v', 'hist_vAng', 'hist_distAng', 'polar' ] }
#     break


f = os.path.join(cut_dir,'trial_data')
# pickle.dump( {'trial_data':trial_data}, bz2.BZ2File(f+'.bz2','w') )
pickle.dump( {'trial_data':trial_data}, open(f+'.pik','wb') )

### Load precomputed analysis results.

In [ ]:
# List previously computed cuts.
cut_dirs = os.listdir(analysis_dir)
print(cut_dirs)
cut_label = cut_dirs[0]

f    = os.path.join(analysis_dir,cut_label,'trial_data')
data = pickle.load( open(f+'.pik','rb') )
globals().update(data)

# Plot

Distributions and other statistical quantities were precomputed in the previous section. Time series and trajectories plots require to reload each trial's trial file, one at a time.

### Trajectories and time series

In [ ]:
# fig_dir = { k:os.path.join(analysis_dir,cut_label,k) for k in 
#                ['trajectories', 'angle-vs-time'] }
# for d in fig_dir.values():
#     if not os.path.exists(d):
#         os.mkdir(d)

# # Creating a new figure for each trial creates a memory leak
# # I haven't been able to plug, so I'm creating one figure for
# # trajectories and one for angles and reusing them.
# fig_traj = plt.figure(figsize=(9,)*2)
# fig_ang  = plt.figure(figsize=(12,6))

# for i,trial_file in enumerate(trials.index):
#     print('\r'+' '*200+'\r'+f'{i+1}/{len(trials)}',end='')
    
#     trial = load_trial(trial_file, load_data=True)
#     trial = compute_kinematics(trial)
#     globals().update(trial)
    
#     # Trajectories.
#     fig,ax = fig_traj,fig_traj.gca()
#     ax.add_patch(plt.Circle(center, R_px, facecolor='None', 
#                                    edgecolor='k', lw=0.5))
#     ax.plot(*np.moveaxis(data[::5,:n_ind,:2],2,0),lw=0.5)
#     ax.axis('equal')
#     ax.yaxis.set_inverted(True)
#     fig.suptitle(trial_name)
#     fig.savefig(os.path.join(fig_dir['trajectories'],trial_name+'.png'))
#     fig.clf()
    
#     # Angle vs time.
#     fig,ax = fig_traj,fig_traj.gca()
#     ax.plot(time[:,None],pos[:,:,2])
#     ax.set_xlabel('Time (s)')
#     ax.set_ylabel('Angle (rad)')
#     fig.suptitle(trial_name)
#     fig.savefig(os.path.join(fig_dir['angle-vs-time'],trial_name+'.png'))
#     fig.clf()
    
# plt.close('all')

### Statistics

In [ ]:
# f    = os.path.join(analysis_dir,cut_label,'trial_data')
# data = pickle.load( open(f+'.pik','rb') )
# globals().update(data)

# fig_dir = { k:os.path.join(analysis_dir,cut_label,k) for k in [ #'valid_fraction', 
#             'hist_dWall', 'hist_v', 'hist_vAng', 'hist_distAng', 'polar' ] }
# for d in fig_dir.values():
#     if not os.path.exists(d):
#         os.mkdir(d)


# fig = plt.figure()
# ax  = fig.gca()

# for i,trial_file in enumerate(trials.index):
#     print('\r'+' '*200+'\r'+f'{i+1}/{len(trials)}',end='')
    
#     globals().update(trials.loc[trial_file].to_dict())
#     globals().update(trial_data[trial_file])
    
# #     # Valid fraction.
# #     bp = plt.bar(*zip(*valid_fraction.items()))
# #     for bar in bp:
# #         h,x,w = bar.get_height(),bar.get_x(),bar.get_width()
# #         plt.annotate(f'{h:.2f}', xy=(x+w/2,1.01), ha='center', va='bottom')
# #     plt.ylim(0,1.1)
# #     plt.ylabel('Valid fraction')
# #     plt.title(cut_label)
# #     plt.suptitle(trial_name)
# #     plt.savefig(os.path.join(fig_dir['valid_fraction'],trial_name+'.png'))
# #     plt.close()
    
#     # Distributions of distance to the wall, speed, and angular speed.
#     H = dict( hist_dWall='d_wall (cm)', hist_v='v (cm/s)', hist_vAng='v_ang (rad/s)' )
#     for name,label in H.items():
#         ax = fig.gca()
#         h,b = locals()[name]
#         ax.bar(b[:-1],h,width=b[1:]-b[:-1])
#         ax.set_yscale('log')
#         ax.set_xlabel(label)
#         ax.set_ylabel('frequency')
#         ax.set_title(cut_label)
#         fig.suptitle(trial_name)
#         fig.savefig(os.path.join(fig_dir[name],trial_name+'.png'))
#         fig.clf()
    
#     # Pairwise distance-angle distribution.
#     if not hist_distAng is None:
#         ax = fig.gca()
#         h,b1,b2 = hist_distAng
#         m = ax.pcolormesh(b1, b2*180/np.pi, h.T, cmap='Oranges')
#         ax.set_xlabel('pair distance (cm)')
#         ax.set_ylabel('pair angle (deg)')
#         fig.colorbar(m)
#         ax.set_title(cut_label)
#         fig.suptitle(trial_name)
#         fig.savefig(os.path.join(fig_dir['hist_distAng'],trial_name+'.png'))
#         fig.clf()
    
#     # Pairwise polar order parameter vs distance.
#     if not polar is None:
#         ax = fig.gca()
#         p,b = polar
#         ax.plot((b[1:]+b[:-1])/2,p,marker='o',mfc='None',ms=4)
#         ax.set_xlabel('pair distance (cm)')
#         ax.set_ylabel('mean cosine of pair angle')
#         ax.set_ylim(-1,1)
#         ax.set_title(cut_label)
#         fig.suptitle(trial_name)
#         fig.savefig(os.path.join(fig_dir['polar'],trial_name+'.png'))
#         fig.clf()
    
# #     if i==10:
# #         break

# plt.close('all')

In [ ]:
# ''' Analyze instances of unusually high velocity. '''

# # At 30 fps, |v_ang|=30 (about where the rare peaks start) 
# # corresponds to about pi/3 in one frame.
# print('v_ang for pi/3 in (1/30) second:',np.pi/3*fps)

# print('Instances of unusually high v_ang:')
# for f in fish_list:
#     ang_diff  = df[f,'ang'].diff()
#     I = np.nonzero(np.absolute(ang_diff.values)>1)[0]
#     for i in I[:5]:
#         display(df[f,'ang'].iloc[i-1:i+2])

# Aggregate

### Area and aspect ratio

In [ ]:
f    = os.path.join(analysis_dir,cut_label,'trial_data')
data = pickle.load( open(f+'.pik','rb') )
globals().update(data)

for pop,files1 in trials.groupby('pop').groups.items():
    fig,ax = plt.subplots(1,2,figsize=(12,3))
    fig.suptitle(pop)
    for age,files2 in trials.loc[files1].groupby('age').groups.items():
        for i,name in enumerate(['hist_area','hist_aspect']):
            b = trial_data[files2[0]][name][1]
            h = np.mean([trial_data[f][name][0] for f in files2],axis=0)
            ax[i].plot(b[:-1],h,label=f'{age}')
        
    for i in range(2):
        ax[i].set_title(['Fish Area','Fish Aspect Ratio'][i])
        ax[i].legend(title='age')
    plt.show()

### Valid fraction

In [ ]:
f    = os.path.join(analysis_dir,cut_label,'trial_data')
data = pickle.load( open(f+'.pik','rb') )
globals().update(data)

grouped_valid_fractions = {}
for g,files in grouped_trials.groups.items():
    grouped_valid_fractions[g] = defaultdict(list)
    for i,f in enumerate(files):
        for cut,vf in trial_data[f]['valid_fraction'].items():
            grouped_valid_fractions[g][cut].append(vf)

''' Boxplot of the fraction of valid frames for each type of invalidity. 
    One boxplot for each type of trial. '''
# for k,valid_fractions in grouped_valid_fractions.items():
#     plt.boxplot(valid_fractions.values(),labels=valid_fractions.keys())
# #     plt.violinplot([valid_fractions[c] for c in cut_names],showextrema=False)
#     plt.xlabel('Cut')
#     plt.ylabel('Valid fraction')
#     plt.ylim(0,1.05)
#     pop,age,n_ind = k
#     name = f'{pop}_{age}dpf_n{n_ind}'
#     plt.title(name)
#     plt.show()

''' Boxplot of the fraction of valid frames for each type of trial. 
    One boxplot for each type of invalidity. '''
# for cut_type in next(iter(grouped_valid_fractions.values())).keys():
#     vf_summary = { f'{k[0]}_{k[1]}dpf_n{k[2]}':vf[cut_type] for k,vf in grouped_valid_fractions.items() }
#     plt.figure(figsize=(12,4))
#     plt.boxplot(vf_summary.values(),labels=vf_summary.keys())
#     plt.ylabel('Valid fraction')
#     plt.ylim(0,1)
#     plt.xticks(rotation=90)
#     plt.title(cut_type)
#     plt.show()

''' Boxplot of the fraction of valid frames for each type of trial. '''
cut_types = next(iter(grouped_valid_fractions.values())).keys()
# for cut_type in cut_types:
for cut_type in ['final']:
    vf_summary = { f'{k[0]}_{k[1]}dpf_n{k[2]}':vf[cut_type] for k,vf in grouped_valid_fractions.items() }
    plt.figure(figsize=(12,4))
    plt.boxplot(vf_summary.values(),labels=vf_summary.keys())
    plt.ylabel('Valid fraction')
    plt.ylim(0,1.02)
    plt.xticks(rotation=90)
    plt.title(f'{cut_type} cut')
    plt.show()

In [ ]:
''' Identify trials with a higher fraction of invalid frames. '''

for f in trials.index:
    vf = trial_data[f]['valid_fraction']
    if vf['final']<0.8:
        print(trials.loc[f]['trial_name'])
        print(', '.join([ f'{k}:{v:.2g}' for k,v in vf.items() ]))
        print()

### Wall distance

In [ ]:
f    = os.path.join(analysis_dir,cut_label,'trial_data')
data = pickle.load( open(f+'.pik','rb') )
globals().update(data)

pops = trials['pop'].unique()
ages = np.sort(trials['age'].unique())
colors = dict(zip( ages, plt.cm.viridis(np.linspace(0,1,len(ages))) ))

for pop in pops:
    fig,axs = plt.subplots(1,2,figsize=(12,4))
    for age in ages:
        for i,n_ind in enumerate([2,5]):
            ax = axs[i]
            files = grouped_trials.get_group((pop,age,n_ind)).index
            h,b = trial_data[files[0]]['hist_dWall']
            b   = b/tank_radius_vs_age[age]
            x   = (b[1:]+b[:-1])/2
            H = np.array([ trial_data[f]['hist_dWall'][0] for f in files ])
            h = np.nansum(H,axis=0)
            h   = h/(2*np.pi*(1-x))
            m = ax.plot(x, h, color=colors[age], label=f'{age}dpf')
    for i,n_ind in enumerate([2,5]):
        ax = axs[i]
#         ax.set_ylim(1e1,None)
        ax.set_xlabel('distance to the wall (tank radii)')
        ax.set_ylabel('density per unit area (arbitrary units)')
        ax.set_yscale('log')
        ax.set_title(f'{pop}_n{n_ind}')
        ax.legend(ncol=2)
    plt.show()
#     break

### Speed distribution

In [ ]:
f    = os.path.join(analysis_dir,cut_label,'trial_data')
data = pickle.load( open(f+'.pik','rb') )
globals().update(data)

pops = trials['pop'].unique()
ages = np.sort(trials['age'].unique())
colors = dict(zip( ages, plt.cm.viridis(np.linspace(0,1,len(ages))) ))

for pop in pops:
    fig,axs = plt.subplots(1,2,figsize=(12,4))
    for age in ages:
        for i,n_ind in enumerate([2,5]):
            ax = axs[i]
            files = grouped_trials.get_group((pop,age,n_ind)).index
            h,b = trial_data[files[0]]['hist_v']
            b   = b/tank_radius_vs_age[age]
            H = np.array([ trial_data[f]['hist_v'][0] for f in files ])
            h = np.nansum(H,axis=0)
            m = ax.plot((b[1:]+b[:-1])/2, h, color=colors[age], label=f'{age}dpf')
    for i,n_ind in enumerate([2,5]):
        ax = axs[i]
        ax.set_xlim(0,3)
        ax.set_xlabel('speed (tank radii/s)')
        ax.set_ylabel('frequency')
        ax.set_yscale('log')
        ax.set_title(f'{pop}_n{n_ind}')
        ax.legend()
    plt.show()
#     break

### Angular speed distribution

In [ ]:
f    = os.path.join(analysis_dir,cut_label,'trial_data')
data = pickle.load( open(f+'.pik','rb') )
globals().update(data)

pops = trials['pop'].unique()
ages = np.sort(trials['age'].unique())
colors = dict(zip( ages, plt.cm.viridis(np.linspace(0,1,len(ages))) ))

for pop in pops:
    fig,axs = plt.subplots(1,2,figsize=(12,4))
    for age in ages:
        for i,n_ind in enumerate([2,5]):
            ax = axs[i]
            files = grouped_trials.get_group((pop,age,n_ind)).index
            h,b = trial_data[files[0]]['hist_vAng']
            H = np.array([ trial_data[f]['hist_vAng'][0] for f in files ])
            h = np.nansum(H,axis=0)
            m = ax.plot((b[1:]+b[:-1])/2, h, color=colors[age], label=f'{age}dpf')
    for i,n_ind in enumerate([2,5]):
        ax = axs[i]
        ax.set_xlabel('speed (rad/s)')
        ax.set_ylabel('frequency')
        ax.set_yscale('log')
        ax.set_title(f'{pop}_n{n_ind}')
        ax.legend()
    plt.show()
#     break

### Joint pair distance-pair angle distribution

In [ ]:
# for (pop,age,n_ind),files in grouped_trials.groups.items():
#     if n_ind==1:
#         continue
#     h,b1,b2 = trial_data[files[0]]['hist_distAng']
#     H = np.array([ trial_data[f]['hist_distAng'][0] for f in files ])
#     h = np.nanmean(H,axis=0)
#     plt.pcolormesh(bins_d, bins_a*180/np.pi, h.T, cmap='Oranges')
#     plt.xlabel('pair distance (cm)')
#     plt.ylabel('pair angle (deg)')
#     plt.colorbar()
#     plt.title(cut_label)
#     plt.suptitle(f'{pop}_{age}dpf_n{n_ind}')
#     plt.show()
# #     break

In [ ]:
pops = trials['pop'].unique()
ages = np.sort(trials['age'].unique())

# for pop in pops:
for pop in ['RC']:
    for age in ages:
        fig,axs = plt.subplots(1,2,figsize=(12,4))
        for i,n_ind in enumerate([2,5]):
            ax = axs[i]
            files = grouped_trials.get_group((pop,age,n_ind)).index
            h,b1,b2 = trial_data[files[0]]['hist_distAng']
            H = np.array([ trial_data[f]['hist_distAng'][0] for f in files ])
            h = np.nanmean(H,axis=0)
            m = ax.pcolormesh(b1, b2*180/np.pi, h.T, cmap='Oranges')
            ax.set_xlabel('pair distance (cm)')
            ax.set_ylabel('pair angle (deg)')
            fig.colorbar(m,ax=ax)
            ax.set_title(f'{pop}_{age}dpf_n{n_ind}')
        plt.show()
#         break
    break

In [ ]:
pops   = trials['pop'].unique()
ages   = np.sort(trials['age'].unique())
colors = dict(zip( ages, plt.cm.viridis(np.linspace(0,1,len(ages))) ))

n_ind  = 5
for pop in pops:
    plt.figure(figsize=(6,4))
    for age in ages:
#         for i,n_ind in enumerate([2,5]):
        files = grouped_trials.get_group((pop,age,n_ind)).index
        p,b = trial_data[files[0]]['polar']
        P = np.array([ trial_data[f]['polar'][0] for f in files ])
        with warnings.catch_warnings():
            p = np.mean(P,axis=0)
        plt.plot( (b[1:]+b[:-1])/2/tank_radius_vs_age[age], p, lw=2, 
                  color=colors[age], label=f'{pop}_{age}dpf_n{n_ind}' )
        plt.xlabel('pair distance/tank radius')
        plt.ylabel('mean cosine of pair angle')
        plt.ylim(-1,1)
        plt.title(cut_label)
    #     plt.suptitle(f'{pop}_{age}dpf_n{n_ind}')
        plt.legend(loc='center left', bbox_to_anchor=(1.01,0.5))
    plt.show()